<a href="https://colab.research.google.com/github/aaoiii/2024-ESAA-OB/blob/main/3%EC%A3%BC%EC%B0%A8_%ED%95%B8%EC%A6%88%EC%98%A87%EC%9E%A5_%EC%95%99%EC%83%81%ED%94%8C_%ED%95%99%EC%8A%B5%EA%B3%BC_%EB%9E%9C%EB%8D%A4_%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 7. 앙상플 학습과 랜덤 포레스트

- 앙상블
  - 훈련 세트로부터 무작위로 각기 다른 서브셋을 만들어 일련의 결정트리 분류기 훈련
  - 개별 트리의 예측을 구한 후 가장 많은 선택을 받은 클래스를 예측으로 삼는다
- 랜덤포레스트 : 결정트리 앙상블
- 배깅, 부스팅, 스태킹

## 7.1 투표 기반 분류기

- 각 분류기의 예측을 모아서 가장 많이 선택된 클래스 예측하기
- **직접 투표 분류기**
- 약한 학습기라도 많고 다양하면 앙상블은 강한 학습기가 될 수 있다
  - 큰수의 법칙

In [13]:
import warnings
warnings.filterwarnings('ignore')

# import package

import numpy as np
import os

#5장에서 소개한 moons dataset 불러오기

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X,y = make_moons(n_samples=100, noise=0.15)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf=LogisticRegression()
rnd_clf=RandomForestClassifier()
svm_clf=SVC()

voting_clf=VotingClassifier(
    estimators=[('lr',log_clf),('rf',rnd_clf),('svc',svm_clf)],
    voting='hard')
voting_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [15]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train,y_train)
  y_pred=clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.85
RandomForestClassifier 0.95
SVC 0.95
VotingClassifier 0.95


- 모든 분류기가 클래스의 확률을 예측할 수 있으면 (predict_proba()) 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측할 수 있다

: **간접 투표** soft voting
  - 확률이 높은 투표에 비중을 더 준다 => 직접투표보다 성능이 높다
  - voting='soft'
  - 모든 분류기가 클래스의 확률을 추정할 수 있어야한다
  - SVC에는 probability 매개변수를 True로 지정해야한다

## 7.2 배깅과 부스팅

- **배깅** : 훈련 세트에서 중복을 허용하여 샘플링하는 방식
  - 한 예측기를 위해 같은 훈련 샘플을 여러번 샘플링 할 수 있다
- **페이스팅** : 중복을 허용하지 않고 샘플링

- 모든 예측기가 훈련을 마치면 앙상블은 모든 예측을 모아서 새로운 샘플에 대한 예측을 만든다
  - 분류 : 통계적 최빈값
  - 회귀 : 평균
  - 개별예측기가 수집 함수를 통과하면 편향과 분산이 감소한다
  - 앙상블 결과는 원본 데이터셋으로 하나의 예측기 훈련시킬 때랑 편향은 비슷하지만 분산은 줄어든다


### 7.2.1 사이킷런의 배깅과 부스팅

- BaggingClassifier(BaggingRegressor)
- 페이스팅 사용 : bootstrap=False
- n_jobs : CPU코어 수 지정 (-1은 모든 코어 사용)

In [16]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf=BaggingClassifier(
    DecisionTreeClassifier(),n_estimators=500,
    max_samples=50, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred=bag_clf.predict(X_test)


- BaggingClassifier은 기반이 되는 분류기가 predict_proba()함수가 있으면 간접 투표 방식을 사용한다
- 부트스트래핑 : 각 예측기가 학습하는 서브셋에 다양성을 증가시키므로 편향이 조금 높다
but 분산을 감소시킨다

### 7.2.2 oob 평가

- 선택되지 않은 훈련샘플 oob 샘플
- 앙상블의 평가는 각 예측기의 oob 평가를 평균하여 얻는다
- oob_score=True


In [17]:
bag_clf=BaggingClassifier(
    DecisionTreeClassifier(),n_estimators=500,
    bootstrap=True,n_jobs=-1,oob_score=True)
bag_clf.fit(X_train,y_train)
bag_clf.oob_score_

0.9

In [18]:
from sklearn.metrics import accuracy_score
y_pred=bag_clf.predict(X_test)
accuracy_score(y_test,y_pred)

0.95

- oob_decision_function : 결정함수는 각 훈련 샘플의 클래스 확률을 반환

In [19]:
bag_clf.oob_decision_function_

array([[1.        , 0.        ],
       [1.        , 0.        ],
       [0.0106383 , 0.9893617 ],
       [0.00574713, 0.99425287],
       [0.0867052 , 0.9132948 ],
       [0.13043478, 0.86956522],
       [0.92696629, 0.07303371],
       [0.98369565, 0.01630435],
       [0.42328042, 0.57671958],
       [0.87570621, 0.12429379],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.01069519, 0.98930481],
       [0.03409091, 0.96590909],
       [0.65174129, 0.34825871],
       [0.33333333, 0.66666667],
       [0.1043956 , 0.8956044 ],
       [0.98245614, 0.01754386],
       [0.98876404, 0.01123596],
       [0.94623656, 0.05376344],
       [1.        , 0.        ],
       [0.10404624, 0.89595376],
       [0.99468085, 0.00531915],
       [0.01098901, 0.98901099],
       [0.51428571, 0.48571429],
       [1.        , 0.        ],
       [0.88481675, 0.11518325],
       [0.04069767, 0.95930233],
       [1.        , 0.        ],
       [0.00558659, 0.99441341],
       [0.

## 7.3 랜덤 패치와 랜덤 서브스페이스

- BaggingClassifier은 특성 샘플링 지원
  - max_features, bootstrap_features
  - 특성에 대한 샘플링
  - 각 예측기는 무작위로 선택한 입력 특성의 일부분으로 훈련

- 랜덤 패치 방식 : 훈련 특성과 샘플을 모두 샘플링
- 랜덤 서브스페이스 방식 : 훈련샘플 모두 사용, bootstrap=False,max_samples=1.0
- 특성 샘플링은 더 다양한 예측기를 만들며 편향을 늘리는 대신 분산을 낮춘다

## 7.4 랜덤 포레스트

- 배깅 or 페이스팅을 적용한 결정 트리 앙상블
- max_samples : 훈련 세트의 크기 지정
- RandomForestClassifier

In [20]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf=RandomForestClassifier(n_estimators=500,max_leaf_nodes=16,n_jobs=-1)
rnd_clf.fit(X_train,y_train)
y_pred_rf=rnd_clf.predict(X_test)

- 트리의 노드를 분할할 때 전체 특성 중에서 최선의 특성을 찾는 대신 무작위로 선택한 특성 후보 중에서 최적의 특성을 찾는 식으로 무작위성 주입한다

- 편향을 손해보는 대신 분산을 낮춘다

In [22]:
bag_clf=BaggingClassifier(
    DecisionTreeClassifier(max_features='auto',max_leaf_nodes=16),
    n_estimators=500,max_samples=1.0,bootstrap=True, n_jobs=-1)

### 7.4.1 엑스트라 트리

- 트리를 무작위하게 만들기 위해 최적의 임곗값을 찾는 대신 후보 특성을 사용해 무작위로 분할한 다음 그중 최상의 분할을 선택한다
- **익스트림 랜덤 트리**
- 편향이 늘어나지만 분산을 낮춘다
- 빠르다

### 7.4.2 특성중요도

- 랜덤포레스트 => 특성의 상대적 중요도를 측정하기 쉽다
- 얼마나 불순도를 감소시키는지
- 가중치 평균, 각 노드의 가중치는 연관된 훈련 샘플 수와 같다
- feature_importances_

In [23]:
from sklearn.datasets import load_iris
iris=load_iris()
rnd_clf=RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris['data'], iris['target'])
for name, score in zip(iris['feature_names'],rnd_clf.feature_importances_):
  print(name,score)

sepal length (cm) 0.09439859547321541
sepal width (cm) 0.023919707646872585
petal length (cm) 0.46445279970833564
petal width (cm) 0.41722889717157624


## 7.5 부스팅

- 약한 학습기를 여러개 연결하여 강한 학습기를 만드는 앙상블 방법
- 앞의 모델을 보완해나가면서 예측기 학습
- 에이다 부스트, 그레이디언트 부스팅

### 7.5.1 에이다부스트

- 이전모델이 과소적합했던 훈련 샘플의 가중치를 높이는 것
- 새로운 예측기는 학습하기 어려운 샘플에 점점 맞춰진다
- 잘못 분류된 훈련 샘플의 가중치를 상대적으로 높여서 훈련시킨다
- 각 샘플 가중치 w는 초기에 1/m으로 초기화
- 예측기가 정확할수록 가중치가 높아지게 된다
  무작위로 예측하는 정도라면 가중치는 0에 가깝다


- SAMME 라는 에이다부스트의 다중 클래스 버전을 사용
- 클래스가 2개뿐이라면 SAMME는 에이다부스트와 동일
- SAMME.R : 예측기에 predict_proba()가 있다면

In [24]:
# max_depth=1 : 결정노드 하나와 리프노드 두개
from sklearn.ensemble import AdaBoostClassifier

ada_clf=AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),n_estimators=200,
    algorithm='SAMME.R', learning_rate=0.5)
ada_clf.fit(X_train,y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

### 7.5.2. 그레디언트 부스팅

- 이전까지의 오차를 보정하도록 예측기를 순차적으로 추가 + 이전 예측이 만든 잔여 오차에 새로운 예측기 학습시

In [25]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1=DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

DecisionTreeRegressor(max_depth=2)

In [26]:
y2=y-tree_reg1.predict(X)
tree_reg2=DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y2)

DecisionTreeRegressor(max_depth=2)

In [29]:
y3=y2-tree_reg1.predict(X)
tree_reg3=DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X,y3)

DecisionTreeRegressor(max_depth=2)

In [31]:
#y_pred=sum(tree.predict(X_new) for tree in (tree_reg1,tree_reg2, tree_reg3))

In [32]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt=GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

- learning_rate : 각 트리의 기여 정도 조절
- 0.1처럼 낮게 설정하면 많은 트리가 필요하지만 예측 성능은 좋아진다
: 축소

- staged_predict() : 최적의 트리 수를 찾기 위해 조기종료 기

In [33]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val=train_test_split(X,y)

gbrt=GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors=[mean_squared_error(y_val, y_pred) for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators=np.argmin(errors)+1

gbrt_best=GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=50)

- warm_start=True : fit() 메서드가 호출될 때 기존 트리를 유지하고 훈련을 추가할 수 있도록 한다

In [34]:
gbrt=GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error=float('inf')
error_going_up=0
for n_estimators in range(1,120):
  gbrt.n_estimators=n_estimators
  gbrt.fit(X_train, y_train)
  y_pred=gbrt.predict(X_val)
  val_error=mean_squared_error(y_val, y_pred)
  if val_error < min_val_error:
    min_val_error = val_error
    error_going_up=0
  else:
    error_going_up+=1
    if error_going_up == 5:
      break #조기종료


- subsample : 각 트리가 훈련할 때 사용할 훈련 샘플의 비율 지정
- 확률적 그레디언트 부스팅 (편향 up 분산 down)

In [36]:
# 최적화된 그레디언트 부스팅 구현
import xgboost

xgb_reg=xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred=xgb_reg.predict(X_val)

In [37]:
xgb_reg.fit(X_train, y_train, eval_set=[(X_val, y_val)],early_stopping_rounds=2)
y_pred=xgb_reg.predict(X_val)

[0]	validation_0-rmse:0.38538
[1]	validation_0-rmse:0.30644
[2]	validation_0-rmse:0.26933
[3]	validation_0-rmse:0.23813
[4]	validation_0-rmse:0.22133
[5]	validation_0-rmse:0.21339
[6]	validation_0-rmse:0.20932
[7]	validation_0-rmse:0.20727
[8]	validation_0-rmse:0.20712
[9]	validation_0-rmse:0.20726


## 7.6 스태킹

- 앙상블에 속한 모든 예측기의 예측을 취합하는 간단한 함수를 사용하는 대신 취합하는 모델을 훈련
- 마지막 예측기 : 블렌더, 메타학습기
- 블렌더 학습 : 홀드아웃 세트 이용
  - 훈련 세트를 두개의 서브셋으로 나눔
